## GLM implimentation from first principles ##
In this notebook I compare the results obtained from a piece of code written based on first principles and statsmodels implimentation of GLMs
### GLM Linear Regression ###

In [1]:
def glm_fit(X,Y):
    y=Y
    beta=np.zeros(X.shape[1])
    H=np.dot(X.T,X)
    log_like_hist=[]
    Converged=False
    for i in range(29):
        nabla=np.dot(np.matmul(X.T,X),beta)-np.dot(X.T,y)
        beta=beta-np.dot(np.linalg.inv(H),nabla)
    log_lik_1=np.dot(np.transpose(y-np.dot(X,beta)),(y-np.dot(X,beta)))
    sigma_2=log_lik_1/(X.shape[0]-X.shape[1])
    log_lik_constants=-0.5*X.shape[0]*np.log(2*np.pi)-X.shape[0]*np.log(np.sqrt(sigma_2))
    log_lik=log_lik_constants-(0.5*log_lik_1)/(sigma_2)
    SE=np.diag(np.sqrt(np.abs(np.linalg.inv(H)*sigma_2)))
        
    return {'Coefficients':beta,'Log_Lik':log_lik,'Std.Err':SE}

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import os

In [5]:
os.chdir("/Users/gunnvantsaini/Documents/Ebooks/Programming and Statistical Packages/R/Data")
data=pd.read_csv("HousePrices.csv")
data['ones']=np.ones(data.shape[0])
X=data[['ones',"SqFt","Bedrooms"]]
y=data["Price"]

In [7]:
glm_fit(X,y)

{'Coefficients': array([ -6367.59669871,     49.49885995,  12486.05779548]),
 'Log_Lik': -1454.6301249455346,
 'Std.Err': array([  1.78279148e+04,   1.01122638e+01,   2.94713432e+03])}

In [8]:
'''
Check the results using standard glm implimentation

'''i
mport statsmodels.formula.api as smf
import statsmodels.api as sm
mod=smf.glm("Price~SqFt+Bedrooms",data=data,family=sm.families.Gaussian()).fit()
print mod.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Price   No. Observations:                  128
Model:                            GLM   Df Residuals:                      125
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                   445254299.235
Method:                          IRLS   Log-Likelihood:                -1454.6
Date:                Tue, 03 Jan 2017   Deviance:                   5.5657e+10
Time:                        23:40:40   Pearson chi2:                 5.57e+10
No. Iterations:                     4                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept  -6367.5967   1.78e+04     -0.357      0.721     -4.13e+04  2.86e+04
SqFt          49.4989     10.112      4.895      0.0

### GLM Logistic ###

In [9]:
def glm_logistic(X,y):
    beta=np.zeros(X.shape[1])
    for i in range(29):
        p=1/(1+np.exp(np.dot(-1*X,beta)))
        nabla=np.dot(X.T,(p-y))
        B=np.diag(p*(1-(1*p)))
        H=np.dot(np.dot(X.T,B),X)
        beta=beta-np.dot(np.linalg.inv(H),nabla)
    log_lik=np.dot(y,np.log(p))+np.dot((1-y),np.log(1-p))
    SE=np.diag(np.sqrt(np.abs(np.linalg.inv(H))))
    return {'Coefficients':beta,'Log_lik':log_lik,'Std.Err':SE}


In [10]:
data=pd.read_csv("DeathPenalty.csv")
data['ones']=np.ones(data.shape[0])
X=data[['ones','Agg','VRace']]
y=data['Death']

In [12]:
glm_logistic(X,y)   

{'Coefficients': array([-6.67597507,  1.53966135,  1.81064663]),
 'Log_lik': -56.738362454825115,
 'Std.Err': array([ 0.75744456,  0.18672636,  0.53611602])}

In [13]:
'''
Check the results using standard glm implimentation

'''
mod=smf.glm('Death~Agg+VRace',data=data,family=sm.families.Binomial()).fit()

mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Death   No. Observations:                  362
Model:                            GLM   Df Residuals:                      359
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -56.738
Date:                Tue, 03 Jan 2017   Deviance:                       113.48
Time:                        23:42:02   Pearson chi2:                     340.
No. Iterations:                     9                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -6.6760      0.757     -8.814      0.000        -8.161    -5.191
Agg            1.5397      0.187      8.246      0.000         1.174     1.906
VRace          1.8106      0.536      3.377      0.001         0.760     2.861
==============================================================================
"""

### GLM Poisson ###

In [14]:
def glm_poison(X,y):
    beta=np.zeros(X.shape[1])
    for i in range(29):
        mean=np.exp(np.dot(X,beta))
        nabla=np.dot(X.T,(y-mean))
        B=np.diag(mean)
        H=-np.dot(np.dot(X.T,B),X)
        beta=beta-np.dot(np.linalg.inv(H),nabla)
    SE=np.diag(np.sqrt(np.abs(np.linalg.inv(H))))
    return {'Coefficient':beta,'Std.Err':SE}


In [15]:
data=pd.read_csv('poisson_sim.csv')
data['ones']=np.ones(data.shape[0])
X=data[['ones','prog','math']]
y=data['num_awards']

In [16]:
glm_poison(X,y)

{'Coefficient': array([-5.57805691,  0.12327256,  0.086121  ]),
 'Std.Err': array([ 0.67682258,  0.16326106,  0.00958606])}

In [17]:
'''
Check the results using standard glm implimentation

'''
mod=smf.glm('num_awards~prog+math',data=data,family=sm.families.Poisson()).fit()
print mod.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:             num_awards   No. Observations:                  200
Model:                            GLM   Df Residuals:                      197
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -189.75
Date:                Tue, 03 Jan 2017   Deviance:                       203.45
Time:                        23:43:33   Pearson chi2:                     227.
No. Iterations:                     8                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -5.5781      0.677     -8.242      0.000        -6.905    -4.252
prog           0.1233      0.163      0.755      0.4